In [1]:
import shutil
import os.path
import xarray as xr
from tests.helpers import make_test_dataset
from zappend.api import zappend

In [2]:
slice_paths = []
for index in range(10):
    slice_path = f"testdata/slice-{index}.zarr" 
    slice_paths.append(slice_path)
    make_test_dataset(uri=slice_path, 
                      shape=(1, 200, 200), 
                      chunks=(1, 200, 200),
                      index=index)

In [3]:
slice_ds = xr.open_zarr("testdata/slice-0.zarr")
slice_ds

<xarray.Dataset>
Dimensions:  (time: 1, y: 200, x: 200)
Coordinates:
  * time     (time) datetime64[ns] 2024-01-01
  * x        (x) float64 0.0 0.005025 0.01005 0.01508 ... 0.9899 0.995 1.0
  * y        (y) float64 0.0 0.005025 0.01005 0.01508 ... 0.9899 0.995 1.0
Data variables:
    chl      (time, y, x) float32 dask.array<chunksize=(1, 200, 200), meta=np.ndarray>
    tsm      (time, y, x) float32 dask.array<chunksize=(1, 200, 200), meta=np.ndarray>

In [4]:
slice_ds.time.encoding

{'chunks': (1,),
 'preferred_chunks': {'time': 1},
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 'units': 'days since 2024-01-01 00:00:00',
 'calendar': 'proleptic_gregorian',
 'dtype': dtype('int64')}

In [5]:
target_path = "testdata/target.zarr"
if os.path.exists(target_path):
    shutil.rmtree(target_path)

In [6]:
zappend(slice_paths, 
        target_uri="testdata/target.zarr", 
        variables={
            "time": {
                "encoding": {
                    "chunks": [100]
                }
            }
        })

In [7]:
target_ds = xr.open_zarr("testdata/target.zarr")
target_ds

<xarray.Dataset>
Dimensions:  (time: 10, y: 200, x: 200)
Coordinates:
  * time     (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2024-01-10
  * x        (x) float64 0.0 0.005025 0.01005 0.01508 ... 0.9899 0.995 1.0
  * y        (y) float64 0.0 0.005025 0.01005 0.01508 ... 0.9899 0.995 1.0
Data variables:
    chl      (time, y, x) float32 dask.array<chunksize=(1, 200, 200), meta=np.ndarray>
    tsm      (time, y, x) float32 dask.array<chunksize=(1, 200, 200), meta=np.ndarray>

In [8]:
target_ds.time

<xarray.DataArray 'time' (time: 10)>
array(['2024-01-01T00:00:00.000000000', '2024-01-02T00:00:00.000000000',
       '2024-01-03T00:00:00.000000000', '2024-01-04T00:00:00.000000000',
       '2024-01-05T00:00:00.000000000', '2024-01-06T00:00:00.000000000',
       '2024-01-07T00:00:00.000000000', '2024-01-08T00:00:00.000000000',
       '2024-01-09T00:00:00.000000000', '2024-01-10T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2024-01-10

In [9]:
target_ds.time.encoding

{'chunks': (100,),
 'preferred_chunks': {'time': 100},
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 'units': 'days since 2024-01-01',
 'calendar': 'proleptic_gregorian',
 'dtype': dtype('int64')}